Example Python Jupyter Notebook that demonstrates:
- Connecting to a Microsoft SQL Server
- Creating dataframes from reading csv files
- Inserting these dataframes into an SQL server database
- Reading the new data back from the database into new dataframes to demonstrate it has been inserted correctly

In [1]:
# Import required modules
import pandas as pd
import pyodbc
import sqlalchemy
from sqlalchemy import create_engine
import urllib

In [2]:
filename = "C:\\Users\\Daniel Underwood\\Documents\\NFL_Database\\TeamsInfo.csv"
teams_df = pd.read_csv(filename)
teams_df.head()

,TeamID,ShortName,FullName,Nickname,Conference,Division,FoundingYear,Colour
0,T01,ARI,Arizona Cardinals,Cardinals,NFC,NFC West,1920,153;0;51
1,T02,ATL,Atlanta Falcons,Falcons,NFC,NFC South,1965,204;0;102
2,T03,BAL,Baltimore Ravens,Ravens,AFC,AFC North,1996,102;0;102
3,T04,BUF,Buffalo Bills,Bills,AFC,AFC East,1960,0;0;255
4,T05,CAR,Carolina Panthers,Panthers,NFC,NFC South,1993,0;153;204


In [3]:
filename = "C:\\Users\\Daniel Underwood\\Documents\\NFL_Database\\GamesInfo.csv"
games_df = pd.read_csv(filename)
games_df.head()

,GameYear,GameWeek,AwayTeam,HomeTeam,NeutralVenue,AwayScore,HomeScore
0,2002,1,SF,NYG,N,16,13
1,2002,1,DET,MIA,N,21,49
2,2002,1,IND,JAC,N,28,25
3,2002,1,MIN,CHI,N,23,27
4,2002,1,BAL,CAR,N,7,10


In [4]:
filename = "C:\\Users\\Daniel Underwood\\Documents\\NFL_Database\\OddsInfo.csv"
odds_df = pd.read_csv(filename)
odds_df.head()

,GameID,OddsDateTime,AwayOddsFrac,HomeOddsFrac
0,G20061MIAPIT,2018-08-23 20:55:44,1.05,0.80
1,G20061CINKC,2018-08-23 20:55:45,1.10,0.77
2,G20061BALTB,2018-08-23 20:55:45,1.45,0.63
3,G20061PHIHOU,2018-08-23 20:55:45,0.33,2.50
4,G20061DENLAR,2018-08-23 20:55:46,0.50,1.75


In [5]:
#Connect to database 'NFL' using SQlAlchemy
connection_str = "DRIVER={SQL SERVER};SERVER=LAPTOP-BJU2VQE0\SIMPLESERVER;DATABASE=NFL;TRUSTED_CONNECTION=YES"
params = urllib.parse.quote_plus(connection_str)
engine = create_engine('mssql+pyodbc:///?odbc_connect=%s' % params)
conn = engine.connect()

In [6]:
teams_df.to_sql('Teams', conn, if_exists='append', index=False)

-1

In [7]:
teams_read_df = pd.read_sql('SELECT * From Teams',conn)
teams_read_df.head()

,TeamID,ShortName,FullName,Nickname,Conference,Division,FoundingYear,Colour
0,T01,ARI,Arizona Cardinals,Cardinals,NFC,NFC West,1920,153;0;51
1,T02,ATL,Atlanta Falcons,Falcons,NFC,NFC South,1965,204;0;102
2,T03,BAL,Baltimore Ravens,Ravens,AFC,AFC North,1996,102;0;102
3,T04,BUF,Buffalo Bills,Bills,AFC,AFC East,1960,0;0;255
4,T05,CAR,Carolina Panthers,Panthers,NFC,NFC South,1993,0;153;204


In [8]:
games_df.to_sql('Games', conn, if_exists='append', index=False)

-1

In [9]:
games_read_df = pd.read_sql('SELECT * From Games',conn)
games_read_df.head()

,GameID,YearWeek,GameYear,GameWeek,AwayTeam,HomeTeam,NeutralVenue,AwayScore,HomeScore,Winner
0,G200210ATLPIT,2002-10,2002,10,ATL,PIT,N,34,34,TIE
1,G200210CINBAL,2002-10,2002,10,CIN,BAL,N,27,38,BAL
2,G200210DETGB,2002-10,2002,10,DET,GB,N,14,40,GB
3,G200210HOUTEN,2002-10,2002,10,HOU,TEN,N,10,17,TEN
4,G200210INDPHI,2002-10,2002,10,IND,PHI,N,35,13,IND


In [10]:
odds_df.to_sql('Odds', conn, if_exists='append', index=False)

-1

In [11]:
odds_read_df = pd.read_sql('SELECT * From Odds',conn)
odds_read_df.head()

,OddsID,GameID,OddsDateTime,AwayOddsFrac,HomeOddsFrac
0,OG200610BALTENAug 23 2018 8:56PM,G200610BALTEN,2018-08-23 20:56:33,0.31,2.50
1,OG200610BUFINDAug 23 2018 8:56PM,G200610BUFIND,2018-08-23 20:56:31,5.25,0.14
2,OG200610CHINYGAug 23 2018 8:56PM,G200610CHINYG,2018-08-23 20:56:37,1.00,0.83
3,OG200610CLEATLAug 23 2018 8:56PM,G200610CLEATL,2018-08-23 20:56:35,2.90,0.28
4,OG200610DALARIAug 23 2018 8:56PM,G200610DALARI,2018-08-23 20:56:36,0.33,2.40


In [12]:
conn.close()